In [1]:
import pandas as pd
import requests
from utils import connect_to_db, load_to_sql

In [7]:
# Obtener datos de https://random-data-api.com/api/v2/beers?size=10

r = requests.get(
    'https://random-data-api.com/api/v2/beers',
    params={'size': 10}
    )

if r.status_code == 200:
    beers_data = r.json()

In [8]:
df_beers = pd.json_normalize(beers_data)
df_beers.head()

,id,uid,brand,name,style,hop,yeast,malts,ibu,alcohol,blg
0,5367,736355c5-f2c6-424e-a599-cfdeae9a7c79,Blue Moon,La Fin Du Monde,Bock,Chelan,1056 - American Ale,Special roast,74 IBU,6.1%,5.3°Blg
1,7321,dfbb81f4-1b42-4fc8-9ff1-43226eccf7c7,Stella Artois,La Fin Du Monde,Belgian Strong Ale,Millennium,1318 - London Ale III,Special roast,88 IBU,7.3%,15.9°Blg
2,6663,929e902e-f63e-4ddb-a937-5b7af5624d48,Dos Equis,Hercules Double IPA,Bock,Olympic,2308 - Munich Lager,Victory,52 IBU,8.1%,5.8°Blg
3,3664,da1e15db-4c33-4f14-b276-12bbfea8ff2d,Lowenbrau,St. Bernardus Abt 12,Wood-aged Beer,Hallertau,3463 - Forbidden Fruit,Vienna,66 IBU,4.0%,12.4°Blg
4,3737,9447b758-6f03-4b63-a087-ddf6372697a4,Lowenbrau,Chocolate St,Strong Ale,Centennial,1275 - Thames Valley Ale,Carapils,16 IBU,3.2%,6.2°Blg


In [5]:
engine = connect_to_db("config.ini", "redshift")

2024-02-06 20:25:49,053 - INFO - Conectándose a la base de datos...
2024-02-06 20:25:49,128 - INFO - Conexión a la base de datos establecida exitosamente


In [9]:
with engine.connect() as conn, conn.begin():

    conn.execute("TRUNCATE TABLE stg_beer")

    load_to_sql(df_beers, "stg_beer", conn, "append")

    conn.execute("""
        MERGE INTO dim_beer
        USING stg_beer
        ON dim_beer.id = stg_beer.id
        WHEN MATCHED THEN
            UPDATE SET
                name = stg_beer.name,
                style = stg_beer.style,
                brand = stg_beer.brand,
                updated_at = GETDATE() 
        WHEN NOT MATCHED THEN
            INSERT (id, uid, brand, name, style, hop, malts, alcohol, blg, created_at, updated_at)
            VALUES (stg_beer.id, stg_beer.uid, stg_beer.brand, stg_beer.name, stg_beer.style, stg_beer.hop, stg_beer.malts, stg_beer.alcohol, stg_beer.blg, GETDATE(), GETDATE())
        """)

2024-02-06 20:36:02,953 - INFO - Cargando datos en la base de datos...
2024-02-06 20:36:03,440 - INFO - Datos cargados exitosamente en la base de datos


In [11]:
engine = connect_to_db("config.ini", "redshift")
with engine.connect() as conn, conn.begin():
    conn.execute("""
        MERGE INTO dim_beer
        USING stg_beer
        ON dim_beer.id = stg_beer.id
        WHEN MATCHED THEN
            UPDATE SET
                name = stg_beer.name,
                style = stg_beer.style,
                brand = stg_beer.brand,
                updated_at = GETDATE() 
        WHEN NOT MATCHED THEN
            INSERT (id, uid, brand, name, style, hop, malts, alcohol, blg, created_at, updated_at)
            VALUES (stg_beer.id, stg_beer.uid, stg_beer.brand, stg_beer.name, stg_beer.style, stg_beer.hop, stg_beer.malts, stg_beer.alcohol, stg_beer.blg, GETDATE(), GETDATE())
        """)

2024-02-06 20:46:16,236 - INFO - Conectándose a la base de datos...
2024-02-06 20:46:16,239 - INFO - Conexión a la base de datos establecida exitosamente
